<a href="https://colab.research.google.com/github/meghasyam/python/blob/master/closuredecorator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def outer_function():
    a = 5  # local to outer_function 
    def inner_function():
        nonlocal a
        a = 10
        print("Inner function: ",a)
    inner_function()
    print("Outer function: ",a)

outer_function()

In [3]:
def print_msg(msg):
# This is the outer enclosing function
    def printer():
# This is the nested function
        nonlocal msg
        msg = "new"
        print(msg)
    printer()

# We execute the function
# Output: Hello
print_msg("Hello")

new


In [0]:
def print_msg(msg):
# This is the outer enclosing function

    def printer():
# This is the nested function
        print(msg)

    return printer  # this got changed

# Now let's try calling this function.
# Output: Hello
another = print_msg("Hello")
another()

In [4]:
def make_multiplier_of(n):
    def multiplier(x):
        return x * n
    return multiplier  #inner functions returns

# Multiplier of 3
times3 = make_multiplier_of(3)

# Multiplier of 5
times5 = make_multiplier_of(5)

# Output: 27
print(times3(9))

# Output: 15
print(times5(3))

# Output: 30
print(times5(times3(2)))

27
15
30


In [0]:
make_multiplier_of.__closure__

In [6]:
times3.__closure__

(<cell at 0x7ff79364bf48: int object at 0xa68b00>,)

In [7]:
times3.__closure__[0].cell_contents  #oops encapsule - package (data + methoeds)  - data hiding -private __ or _

3

In [8]:
times5.__closure__[0].cell_contents

5

In [10]:
def inc(x):
    return x + 1

def dec(x):
    return x - 1

def operate(func, x):
    result = func(x)
    return result

operate(inc,3)
operate(dec,3)

2

In [11]:
def is_called():
    def is_returned():
        print("Hello")
    return is_returned

new = is_called()

#Outputs "Hello"
new()

Hello


In [12]:
def make_pretty(func):
    def inner():
        print("I got decorated")
        func()
    return inner

def ordinary():
    print("I am ordinary")

x = make_pretty(ordinary)

x()

I got decorated
I am ordinary


In [19]:
def make_pretty(func):
    def inner():
        print("I got decorated")
        func()
    return inner

# ordinary -- make_pretty  -oridnary    
@make_pretty
def ordinary():
    print("I am ordinary")

@make_pretty
def ordinary1():
    print("I am ordinary-1")

@make_pretty
def ordinary2():
    print("I am ordinary-2")    

decor = ordinary
decor()
ordinary1()
ordinary2()

I got decorated
I am ordinary
I got decorated
I am ordinary-1
I got decorated
I am ordinary-2


In [17]:
#print(decor.__closure__[0].cell_contents)
print(ordinary.__closure__)

(<cell at 0x7ff79364be28: function object at 0x7ff792d7c730>,)


In [21]:
def smart_divide(func):  # non local 
   def inner(a,b):
      print("I am going to divide",a,"and",b)
      if b == 0:
         print("Whoops! cannot divide")
         return

      return func(a,b)
   return inner

@smart_divide
def divide(a,b):
    return a/b
print(divide(2,3))  # smart - divide (divide)
divide(2,0)    

I am going to divide 2 and 3
0.6666666666666666
I am going to divide 2 and 0
Whoops! cannot divide


In [22]:
def works_for_all(func):
    def inner(*args, **kwargs):
        print("I can decorate any function")
        return func(*args, **kwargs)
    return inner

@works_for_all
def multi(*args,**kwargs):
    print(args) 
    print(kwargs) 


multi("new","new1","new2","new3",key="23",key1="nnn")    

I can decorate any function
('new', 'new1', 'new2', 'new3')
{'key': '23', 'key1': 'nnn'}


In [23]:
def star(func):
    def inner(*args, **kwargs):
        print("*" * 30)
        func(*args, **kwargs)
        print("*" * 30)
    return inner

def percent(func):
    def inner(*args, **kwargs):
        print("%" * 30)
        func(*args, **kwargs)
        print("%" * 30)
    return inner

@star
@percent
def printer(msg):
    print(msg)
printer("Hello")

******************************
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Hello
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
******************************


In [24]:
def trace(f):
...     "Decorate a function to print a message before and after execution."
...     def traced(*args, **kw):
...         "Print message before and after a function call."
...         print("Entering", f.__name__)
...         result = f(*args, **kw)
...         print("Leaving", f.__name__)
...         return result
...     return traced

@trace
... def myfunc(x, a=None):
...     "Simply prints a message and arguments."
...     print("Inside myfunc")
...     print("x:", x, "a:", a)

myfunc("ONE")

Entering myfunc
Inside myfunc
x: ONE a: None
Leaving myfunc


In [0]:
trace.__name__
myfunc.__name__
myfunc.__doc__

In [0]:
from functools import wraps
>>> def simpledec(f):
...     "A really simple decorator to demonstrate functools.wraps."
...     @wraps(f)
...     def wrapper(arg):
...         print("Calling f with arg", arg)
...         return f(arg)
...     return wrapper
... 
>>> @simpledec
... def f(x):
...     "Simply prints its argument."
...     print("Inside f, arg is", x)
... 
>>> f("Hello")
>>> f.__name__
>>> f.__doc__
>>> 

In [0]:
class ctrace:
...     def __init__(self, f):
...         "__init__ records the passed function for later use in __call__()."
...         self.__doc__ = f.__doc__
...         self.__name__ = f.__name__
...         self.f = f
...     def __call__(self, *args, **kw):
...         "Prints a trace line before calling the wrapped function."
...         print("Called", self.f.__name__)
...         return self.f(*args, **kw)
... 
>>> @ctrace
... def simple(x):
...     "Just prints arg and returns it."
...     print("simple called with", x)
...     return x

simple("class decorator")

In [0]:
def callable(o):
...     return hasattr(o, "__call__")
...
>>> def mtrace(cls1):
...     for key, val in cls1.__dict__.items():
...         if key.startswith("__") and key.endswith("__") or not callable(val):
...             continue
...         setattr(cls1, key, trace(val))
...         #print("Wrapped", key)
...     return cls1
... 
>>> @mtrace  # mtrace class - dull
... class dull:
...     def method1(self, arg):
...         print("Method 1 called with arg", arg)
...     def method2(self, arg):
...         print("Method 2 called with arg", arg)

>>> d = dull()
>>> d.method1("Hello")
>>> d.method2("Goodbye")

In [0]:
>>> def framework(f):  # somefunc
...     f.framework = True #somefunc.fr --- true
...     f.author = "Myself" #somefunc.auth ---- myes
...     return f
... 
>>> @framework
... def somefunc(x):
...     pass
... 
>>> somefunc.framework
>>> somefunc.author

In [0]:
>>> counts = {}
>>> def countable(ftype):
...     "Returns a decorator that counts each call of a function against ftype."
...     def decorator(f):
...         "Decorates a function and to count each call."
...         def wrapper(*args, **kw):
...             "Counts every call as being of the given type."
...             try:
...                 counts[ftype] += 1
...             except KeyError:
...                 counts[ftype] = 1
...             return f(*args, **kw)
...         return wrapper
...     return decorator
... 
>>> @countable("short")
... def f1(a, b=None):
...     print("f1 called with", a, b)
... 
>>> @countable("f2")
... def f2():
...     print("f2 called")
... 
>>> @countable("short")
... def f3(*args, **kw):
...     print("f3 called:", args, kw)
... 
>>> for i in range(10): # 0 - 9 
...     f1(1)
...     f2()
...     f3(i, i*i, a=i)

for k in sorted(counts.keys()):
...     print(k, ":", counts[k])